In [1]:
#Setting Spark with MinIO
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,FloatType
from pyspark.sql.functions import col,to_date,date_format,split,split,trim,upper,regexp_replace
import os
from pyspark.sql import SparkSession
S3_ACCESS_KEY = os.getenv("S3_ACCESS_KEY")
S3_SECRET_KEY = os.getenv("S3_SECRET_KEY")
S3_ENDPOINT = os.getenv("S3_ENDPOINT")

spark = SparkSession.builder \
    .appName("MinIOReader") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .config("spark.hadoop.fs.s3a.endpoint", S3_ENDPOINT) \
    .config("spark.hadoop.fs.s3a.access.key", S3_ACCESS_KEY) \
    .config("spark.hadoop.fs.s3a.secret.key", S3_SECRET_KEY) \
    .config("spark.hadoop.fs.s3a.path.style.access", "true") \
    .getOrCreate()

spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
spark.catalog.clearCache()


25/11/26 19:52:05 WARN Utils: Your hostname, harshithts-HP-Pavilion-Gaming-Laptop-15-ec2xxx resolves to a loopback address: 127.0.1.1; using 192.168.1.4 instead (on interface wlo1)
25/11/26 19:52:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/harshithts/RealTimeData-Pipeline/env/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/harshithts/.ivy2/cache
The jars for the packages stored in: /home/harshithts/.ivy2/jars
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-33831197-779c-4d2e-bfd3-0b6c735dcfa0;1.0
	confs: [default]
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 404ms :: artifacts dl 5ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	org.apache.hadoop#hadoop-aws;3.3.4 from central in [default]
	org.wildfly.openssl#wildfly-openssl;1.0.7.Final from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	-------------------------

In [2]:
#schema defining
Profile_schema = StructType([
    StructField("user_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("country", StringType(), True),
    StructField("account_type", StringType(), True),
    StructField("credit_score", IntegerType(), True),
    StructField("status", StringType(), True),
    StructField("timestamp", StringType(), True)
])

merchant_schema = StructType([
    StructField("merchant_id", StringType(), True),
    StructField("merchant_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("country", StringType(), True),
    StructField("risk_level", StringType(), True),
    StructField("timestamp", StringType(), True)
])
account_schema = StructType([
    StructField("account_id", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("current_balance", FloatType(), True),
    StructField("currency", StringType(), True),
    StructField("last_updated", StringType(), True),
  
])



In [3]:

#reading data from the buckets
BUCKET_NAME = os.getenv("BUCKET_NAME")

profile_df=spark.read.format("json")\
    .schema(Profile_schema)\
    .load(f"s3a://{BUCKET_NAME}/bronze/profiles-folder/*.json")

merchant_df= spark.read.format("json")\
    .schema(merchant_schema)\
    .load(f"s3a://{BUCKET_NAME}/bronze/merchant-folder/*.json")

account_df= spark.read.format("json")\
    .schema(account_schema)\
    .load(f"s3a://{BUCKET_NAME}/bronze/account-folder/*.json")




25/11/26 19:52:20 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [4]:
#coalescing the dataframes to 2 partitions each
profile_df = profile_df.coalesce(2)
merchant_df = merchant_df.coalesce(2)
account_df = account_df.coalesce(2)

print("Number of partitions:", profile_df.rdd.getNumPartitions())
print("Number of partitions:", merchant_df.rdd.getNumPartitions())
print("Number of partitions:", account_df.rdd.getNumPartitions())

partition_data = profile_df.rdd.mapPartitionsWithIndex(
    lambda idx, it: [(idx, list(it))] 
).collect()

for idx, rows in partition_data:
    print(f"\n🧩 Partition {idx}: {len(rows)} sample rows")
    for r in rows:
        print(r)




Number of partitions: 2
Number of partitions: 2
Number of partitions: 2



🧩 Partition 0: 5 sample rows
Row(user_id='U0008', name='Patrick Morrison', country='Palestinian Territory', account_type='Current', credit_score=590, status='suspended', timestamp='2025-01-12T01:34:43.903322')
Row(user_id='U0010', name='Tommy Ramos', country='United Arab Emirates', account_type='Savings', credit_score=507, status='active', timestamp='2025-03-03T20:34:28.576867')
Row(user_id='U0007', name='Timothy Weber', country='Uzbekistan', account_type='Current', credit_score=471, status='suspended', timestamp='2025-04-05T05:29:57.884762')
Row(user_id='U0006', name='Edward Reyes', country='Iceland', account_type='Savings', credit_score=713, status='suspended', timestamp='2025-10-16T00:03:28.351299')
Row(user_id='U0009', name='Benjamin Rhodes', country='Korea', account_type='Current', credit_score=389, status='closed', timestamp='2025-06-11T18:47:59.473427')

🧩 Partition 1: 5 sample rows
Row(user_id='U0001', name='Holly Rodriguez', country='Peru', account_type='Current', credit_scor

In [5]:
#esnure every user_id starts with U 
profile_df = profile_df.filter(col("user_id").startswith("U"))

#transforming the timestamp column to date, month and year for profile dataframe

profile_df = profile_df.withColumn("date", to_date(col("timestamp"), "yyyy-MM-dd"))\
.withColumn("month", date_format("date", "MMMM"))\
.withColumn("year", date_format("date", "yyyy"))\
.withColumn("time",date_format(col("timestamp"), "HH:mm:ss"))\
.withColumn("country",regexp_replace(trim(col("country")), " ", ""))


profile_df = profile_df


#splitted names to first and last names
profile_df = profile_df.withColumn("first_name", split(col("name"), " ").getItem(0))
profile_df = profile_df.withColumn("last_name", split(col("name"), " ",).getItem(1))




#handling duplicates

profile_duplictes = profile_df.groupBy("user_id")\
            .count()\
            .filter("count > 1")

profile_df = profile_df.dropDuplicates(["user_id"])

profile_df = profile_df.select('user_id', 'first_name', 'last_name', 'country', 'account_type', 'credit_score', 'status', 'date', 'month', 'year','time')
profile_df.show()


+-------+----------+---------+--------------------+------------+------------+---------+----------+---------+----+--------+
|user_id|first_name|last_name|             country|account_type|credit_score|   status|      date|    month|year|    time|
+-------+----------+---------+--------------------+------------+------------+---------+----------+---------+----+--------+
|  U0001|     Holly|Rodriguez|                Peru|     Current|         519|   closed|2025-01-31|  January|2025|09:10:11|
|  U0002|     Jason|  Gregory|              Guyana|     Current|         838|   active|2025-10-13|  October|2025|09:08:31|
|  U0003|       Amy|      Roy|            HongKong|     Savings|         384|   closed|2025-10-20|  October|2025|07:24:26|
|  U0004|     Chris|   Wilson|                Oman|     Savings|         568|   active|2025-03-12|    March|2025|12:34:48|
|  U0005|     Marie|   Wilson|               Sudan|     Savings|         522|   closed|2025-09-30|September|2025|06:04:08|
|  U0006|    Edw

In [6]:
#ensurinq that merchant id starts with M
merchant_df = merchant_df.filter(col("merchant_id").startswith("M"))

#duplicate records 
duplicates_key_df = (
    merchant_df.groupBy("merchant_id")
    .count()
    .filter("count > 1")
    .select("merchant_id")
)

#removing the dupicates
merchant_df = merchant_df.dropDuplicates(["merchant_id"])

#trimming all string data columns

def trim_all_string_columns(merchant_df):

    string_cols = [f.name for f in merchant_df.schema.fields if f.dataType.simpleString() == "string"]
    for c in string_cols:
        merchant_df = merchant_df.withColumn(c, trim(col(c)))
    return merchant_df

merchant_df = trim_all_string_columns(merchant_df)

#tranfroming the timestamp column to date month and year for merchant data

merchant_df = merchant_df.withColumn("date", to_date(col("timestamp"), "yyyy-MM-dd"))
merchant_df = merchant_df.withColumn("month", date_format("date", "MMMM"))
merchant_df = merchant_df.withColumn("year", date_format("date", "yyyy"))
merchant_df = merchant_df.select('merchant_id', 'merchant_name', 'category', 'country', 'risk_level', 'date', 'month', 'year')


merchant_df.show()



+-----------+--------------------+----------+-------+----------+----------+--------+----+
|merchant_id|       merchant_name|  category|country|risk_level|      date|   month|year|
+-----------+--------------------+----------+-------+----------+----------+--------+----+
|      M0001|        Collins-Love|Restaurant|    USA|      High|2025-05-21|     May|2025|
|      M0002|      Robinson-Hogan|    Retail|    USA|      High|2025-08-14|  August|2025|
|      M0003|         Rojas-Smith|    Retail| Russia|    Medium|2025-10-01| October|2025|
|      M0004|       Gamble-Weaver|      Fuel| Russia|      High|2025-05-08|     May|2025|
|      M0005|    Middleton-Morton|    Retail|    USA|      High|2025-03-21|   March|2025|
|      M0006|   Singleton-Jenkins|    Retail|  India|       Low|2025-02-10|February|2025|
|      M0007|      Mcintosh-Burns|    Travel|  India|      High|2025-08-12|  August|2025|
|      M0008|Arnold, Harper an...|      Fuel| Russia|    Medium|2025-02-02|February|2025|
|      M00

In [7]:
#account id starts with A and user id starts with U
account_df = account_df.filter(col("account_id").startswith("A") & col("user_id").startswith("U")) 
                       
#extracting date month year from last_updated column
account_df = account_df.withColumn("date", to_date(col("last_updated"), "yyyy-MM-dd"))\
                    .withColumn("month", date_format("date", "MMMM")) \
                    .withColumn("year", date_format("date", "yyyy"))\
                    .withColumn("Time",date_format(col("last_updated"), "HH:mm:ss"))

# #ensuring the userid , accountid , cuurency are Uppercase

account_df = account_df.withColumn("account_id", upper(col("account_id"))) \
                       .withColumn("user_id", upper(col("user_id"))) \
                       .withColumn("currency", upper(col("currency")))
##duplicate datas
duplicate_keys = (
    account_df.groupBy("account_id")
    .count()
    .filter("count > 1")
)
account_df = account_df.dropDuplicates(["account_id"])

#filtering the account data for current balance greater than 0 and not null currency

account_df = account_df.filter((col("current_balance") > 0) & (col("currency").isNotNull()))

account_df.select('account_id', 'user_id', 'current_balance', 'currency', 'date', 'month', 'year','time','last_updated').show()

+----------+-------+---------------+--------+----------+---------+----+--------+--------------------+
|account_id|user_id|current_balance|currency|      date|    month|year|    time|        last_updated|
+----------+-------+---------------+--------+----------+---------+----+--------+--------------------+
|     A0001|  U0001|       29800.61|     INR|2025-03-08|    March|2025|13:52:48|2025-03-08T13:52:...|
|     A0002|  U0002|       94011.69|     INR|2025-07-09|     July|2025|22:34:02|2025-07-09T22:34:...|
|     A0003|  U0003|      130480.28|     INR|2025-07-10|     July|2025|07:15:24|2025-07-10T07:15:...|
|     A0004|  U0004|      172161.48|     INR|2025-05-31|      May|2025|05:37:11|2025-05-31T05:37:...|
|     A0005|  U0005|      138958.44|     INR|2025-03-14|    March|2025|17:27:39|2025-03-14T17:27:...|
|     A0006|  U0006|       84217.01|     INR|2025-11-17| November|2025|08:10:07|2025-11-17T08:10:...|
|     A0007|  U0007|      132478.67|     INR|2025-05-12|      May|2025|04:24:57|20

In [8]:
profile_df.write.format("parquet").mode("append").save(f"s3a://{BUCKET_NAME}/silver/processed-profiles/")
merchant_df.write.format("parquet").mode("append").save(f"s3a://{BUCKET_NAME}/silver/processed-merchants/")
account_df.write.format("parquet").mode("append").save(f"s3a://{BUCKET_NAME}/silver/processed-accounts/")
